In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [3]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  dfa = -3*(a**2) - 0.5*(a**(-0.5)) # derivative of -a^3 + sin(3*b) - 1/c + b^2.5 - a^0.5
  dfb = 3*cos(3*b) + 2.5*(b**1.5) # derivative of -a^3 + sin(3*b) - 1/c + b^2.5 - a^0.5
  dfc = 1/(c**2) # derivative of -a^3 + sin(3*b) - 1/c + b^2.5 - a^0.5
  return [dfa, dfb, dfc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [11]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
h = 0.0000001
a = 2
b = 3
c = 4

fa = -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5
fah = - (a + h)**3 + sin(3*b) - 1.0/c + b**2.5 - (a + h)**0.5
dfa = (fah - fa) / h

fb = -a**3 + sin(3*b) - 1.0/c + b **2.5 - a**0.5
fbh = -a**3 + sin(3*(b + h)) - 1.0/c + (b + h)**2.5 - a**0.5
dfb = (fbh - fb) / h

fc = -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5
fch = -a**3 + sin(3*b) - 1.0/(c + h) + b**2.5 - a**0.5
dfc = (fch - fc) / h
numerical_grad = [dfa, dfb, dfc]


# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553966448771
OK for dim 1: expected 10.25699027111255, yours returns 10.256990385926201
OK for dim 2: expected 0.0625, yours returns 0.06249999628948899


In [12]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.



# -----------
fanh = - (a - h)**3 + sin(3*b) - 1.0/c + b**2.5 - (a - h)**0.5
ndfa = (fah - fanh) / (2*h)

fbnh = -a**3 + sin(3*(b - h)) - 1.0/c + (b - h)**2.5 - a**0.5
ndfb = (fbh - fbnh) / (2*h)

fcnh = -a**3 + sin(3*b) - 1.0/(c - h) + b**2.5 - a**0.5
ndfc = (fch - fcnh) / (2*h)
numerical_grad2 = [ndfa, ndfb, ndfc]



# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553380251014
OK for dim 1: expected 10.25699027111255, yours returns 10.256990252699438
OK for dim 2: expected 0.0625, yours returns 0.06250000073038109


In [34]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out
  
  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  def __radd__(self, other):
    return self + other
  
  def __mul__(self, other):
      other = other if isinstance(other, Value) else Value(other)
      out = Value(self.data * other.data, (self, other), '*')

      def _backward():
        self.grad += other.data * out.grad
        other.grad += self.data * out.grad
      out._backward = _backward

      return out 
  def __rmul__(self, other):
      return self * other
  
  def __neg__(self):
     return self * -1
  
  def exp(self):
      out = Value(exp(self.data), (self,), 'exp')
      
      def _backward():
          self.grad += out.grad * exp(self.data)
      out._backward = _backward
      
      return out 
  
  def log(self):
      out = Value(log(self.data), (self,), 'log')
      
      def _backward():
          self.grad += out.grad * (1 / self.data)
      out._backward = _backward
      
      return out
  
  def __pow__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data ** other.data, (self, other), '**')
    
        def _backward():
            self.grad += out.grad * other.data * (self.data ** (other.data - 1))
            other.grad += out.grad * log(self.data) * self.data ** other.data
        out._backward = _backward
    
        return out
  
  def __truediv__(self, other):
     return self * other**-1
  
  def __rtruediv__(self, other):
     other = other if isinstance(other, Value) else Value(other)
     return other / self

  # ------

  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [35]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [36]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

logits = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)
probs = torch.nn.functional.softmax(logits)
loss = -probs[3].log()
loss.backward()
print(loss.item())

/Users/ayush.c/personal/micrograd/venv/lib/python3.9/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


2.1755154132843018


/var/folders/k9/83zxtw0x5z59nycg5lk7smzr0000gp/T/ipykernel_29822/855125790.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = torch.nn.functional.softmax(logits)
